# Review of Course 1

In this notebook I make a review of all the things and techniques that we learned in course 1

## Model architecture - The Sequential model

In this first course we build a range of neural networks by utilizing the fundamental methods of the TensorFlow/Keras **Sequential** Model! 

### Sequential model - layers

The Sequential model architecture let's us build up a network structure by adding layers on top of each other one by another. We have learned how the Sequential model structure often builds from scratch by starting out by adding a selected number layers. 

The first types of layers we defines are the Dense- and the Flatten layers: 

##### **Dense layer:**

The Dense layer adds a layer of Neurons. Each Dense layer needs an activation functions to tell the neurons what to do. There are a lot of options for choosing activation functions to choose from, here we are mainly using: 

_Relu:_
    If x > 0 then x, else 0
    
_Sigmoid_:
    An S-shaped curved logistic funtion (exp(x) / exp(x) +1) that returns 0 or 1
    
_Softmax_:
    Takes a list of values and scales these so the sum of all elements will equal 1. When applied to model outputs we can think of the scaled values as the probability for that class. This is used for multi-class classfication, when oiutput needs to be classified as 1 out of multiple classes! 

##### **Flatten layer:**

The _Flatten_ layer usually comes as the first layer in the model, as this layer serves the purpose of turning the square matrix image of nxn pixels into a 1-dimensional array which can then be feeded into the model! 

Let's look at a simple example of a simple Model architecture: 

In [ ]:
import tensorflow as tf 

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)    
])

## Model - Compilation - compile and train the model

The next thing to do after having built the model architecture, is to actually build it! 

We build the models by compiling it!

When compiling the model we need to make some choices on what specifications we are going to use for:

_optimizer:_
Here we must specifiy which optimizer to use when building the model. We have broad range of different optimizers available in TF, which is another area to dig into in order to pick which optimizer works best for the model at hand. In this example we are using the **Adam()** optimizer!

_loss:_
We must also specify which type of loss function the model should use. This is again a scientific question that depends on the type of data and model we working with. In this example we use the _sparse_categorical_crossentropy_ loss function

_metrics:_
This setting let's us choose which metric we want to use as our measurement for the performance of the model. Here we again have different choices available, and in this example we use the **accuracy** measurement metric!

In [ ]:
# Let's compile the model:
model.compile(optimizer=tf.optimizers.Adam(),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuray'])

# And then we can actually take the model and build it:
model.fit(training_images, training_labels, epochs=5)

## Model - Evaluation

After training the model we can evaluate the model by using evaluation data 

In [ ]:
# Evaluate the model

model.evaluate(test_images, test_labels)

Some things to notice: 

* What is the effect of adding more neurons in a Dense layer?
    - More calculations = slower!
    - We expect also to get a better accuracy, but gives a deeper (more expensive) network
    - Be aware of the "Law of Diminishing returns! Deeper network not always gives a better accuracy!

* What is the effect of adding more layers to the network?
    - Adding more layers gives a deeper network, this should mean that for more complex data more complex networks are needed, and thus deeper networks should give better accuracy! But again, beware of the "Law of diminishing returns"!

* Effect of adding more epochs!
    - Beware of "overfitting" 
    - In principle more epochs should increase accuracy, but often comes at the price of more     overfitting

* Effect of not Normalizing the data
    - Not normalizing will often make it more difficult for the model to find patterns
    


## Model - Using Callbacks

We have seen how we can use callbacks as a way to control the training sequence of the models. By using a callback function we have seen how we can use a callback to stop the model from training further, once a certain amount of a specified metric is reached! 

An example of creating a callback class was as follows:

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        '''
        This method stops the training after reaching 60 percent accuracy

        Args:
          epochs (integer) - index of epoch 
          logs (dict) - metric results from the training epoch
        '''

        # check accuracy
        if (logs.get('loss') < 0.25):

            print("\nLoss is lower than 0.4 so cancelling training!")
            self.model.stop_training = True 

# Instantiate class 
callbacks = myCallback()

To use the callback function when training the model, we add the callback funtion in the fit() method of the model object!

In [ ]:
# Add a callback to the model object
model.fit(x_train, y_train, epochs=111, callbacks=[callbacks]) 
# Notice how the callback is added to the model object!! 

## Model - Improvement using Convolutions

#### Convolutional Layers:

We use the built-in Conv2D layer type when adding convolutional layers! The parameters we add to this layers can be summarized as:

 1. Number of convolutions: Good to use powers of 2!
 2. Size: often 3x3, 5x5, 7x7 etc.
 3. Activation function: Sigmoid, Relu etc. 
 4. First layer we add the input_shape

Normally we'll always follow a convolutional layer with a MaxPooling layer...

##### MaxPooling2D

A layer that is designed to compress the image, while maintaining the content of the features that were highlighted by convolution! 

By specifying (2,2) for the MaxPooling the effect is to quarter the size of the image! 

Some noticeable effects of convolutions: 

 1. Higher number of convolutions - training time up but accuracy should go up too! 
 2. Removing final convolution - training time up, while accuracy drops! 
 3. Adding more convolutions - training time up, accuracy should go up too! 
 

## Model - Training with ImageDataGenerator

#### Data Preprocessing:

Data generators will read pictures and then convert them to _float32_ tensors and feed them to the model! Generators will yield batches of images and their labels

**Normalization**

We will always want to normalize the data that feeds into a network! 

In ImageDataGenerator we do this by setting the input parameter: _rescale_ when initializing the ImageDataGenerator:

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255)
#Here we normalize the pixels to values between 0-1

**Flow from directory**

the ImageDataGenerator allows us to instantiate generators of augmented image batches via .flow(data,labels) or .flow_from_directory(directory):

In [ ]:
# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    "directory",
    target_size=(300,300), # Here we can set the desired number of pixels we want 
    batch_size=128,    # Here we can set the desired batch size
    class_mode='binary' # This setting depends on the type of data we are trying to model
    )

**Training**

We can now setup the actual training

In [ ]:
history = model.fit(
    train_generator, 
    steps_per_epoch, 
    epochs=15, 
    verbose=1
)

#### ImageDateGenerator with Validation:

Here we can now set up data generators for both training and validation sets: 

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    './horse-or-human/',
    target_size=(300,300),
    batch_size=128,
    # Since we use binary_crossentropy loss, we will need binary labels
    class_mode='binary')

# Flow validation images in batches of 128 using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(
    './validation-horse-or-human/',
    target_size = (300, 300),
    batch_size=32,
    # Since we use binary_crossentropy loss, we need binary labels
    class_mode = 'binary')

And then setup the model for traning with both training and validation data:

In [ ]:
history = model.fit(
    train_generator, 
    steps_per_epoch = 8, 
    epochs=15,
    verbose=1,
    validation_data = validation_generator,
    validation_steps=8
)

#### Effect of Compacted Images:

In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)), # Here setting the smaller number of pixels per picture!
    tf.keras.layers.MaxPooling2D(2,2),

    # This is the second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # This is the third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # ... We can put more conv. layers if we want to see the effect! 

    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

**Preprocess the data when compacting the data**

In [ ]:
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    './horse-or-human/',
    target_size=(150,150), 
    batch_size=128,
    class_mode='binary')

# Flow training images in batches of 128 using train_datagen generator 
validation_generator = validation_datagen.flow_from_directory(
    './validation-horse-or-human/',
    target_size=(150, 150),
    batch_size=32, 
    class_mode='binary'
)